1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
import math 
from scipy.misc import derivative
import timeit

In [2]:
def bin_to_dec_func(x):
    
    x_list=[]
    
    for x in str(x):
        
        x_list.append(int(x))

    add= 0
    
    for i in range(len(x_list)):
        
        a = x_list[i] * (2**(len(x_list)-1 - i))
        
        add= add + a
        
    return add
        
def dec_to_bin_func(x):
    
    return bin(x)

def dec_to_hex_func(x):
    
    return hex(x)
    
def hex_to_dec_func(x):
    
    return int(x, 16)

print(bin_to_dec_func(101))

print(dec_to_bin_func(5))

print(hex_to_dec_func('2f'))

print(dec_to_hex_func(3434))

5
0b101
47
0xd6a


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [3]:
def convert_func(x):
    
    bit_list =[]
    
    for x in str(x):
        
        bit_list.append(int(x))
    
    if bit_list[0] == 0:
        
        sign = +1
        
    else:
        
        sign= -1
    
    Mantissa_list= bit_list[9:] 
    
    Exponent_list = bit_list[1:9] 

    exponent=0
    
    for i in range(len(Exponent_list)):
 
        exponent = exponent + ((2**(len(Exponent_list)-1-i)))*Exponent_list[i]
        
    mantissa=0
    
    for i in range(len(Mantissa_list)):
        
        mantissa = mantissa + Mantissa_list[i]*(2**(-i-1))
        
    mantissa=mantissa + 1
    
    return sign * mantissa * (2**(exponent-127))  
    
print(convert_func(10010010011000000000000000000000))

-7.068193710780266e-28


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [4]:
a=1    

while(a!=0):
    
    saved_a=a
    
    a=a/2     
    
print(saved_a)

a=1

while(a!=math.inf): 
    
    saved_a=a     
    
    a=a*2
    
print(saved_a)

5e-324


KeyboardInterrupt: 

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [ ]:
a=1
b=1
dx=0.5

while(a!=b): 
    
    saved_a = a       
    
    a=a+dx                   
    
    dx=dx/2
    
Machine_prec= b-saved_a

print(Machine_prec)

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
def S1_func(a,b,c):
    
    x1= (-b + math.sqrt(b**2 - 4*a*c))/(2*a)
    
    x2= (-b - math.sqrt(b**2 - 4*a*c))/(2*a)
    
    return x1,x2


def S2_func(a,b,c):
    
    x1 = ((-b + math.sqrt(b**2 - 4*a*c)) * (-b - math.sqrt(b**2-4*a*c))) / ((-b - math.sqrt(b**2 - 4*a*c))*(2*a))
    
    x2 = ((-b - math.sqrt(b**2 - 4*a*c)) * (-b + math.sqrt(b**2-4*a*c))) / ((-b + math.sqrt(b**2 - 4*a*c))*(2*a))
    
    return x1,x2

print(S1_func(0.001, 1000, 0.001))

print(S2_func(0.001, 1000, 0.001))

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
def f(x):
    
    return x*(x-1)

def der_func(f,x,d):    
    
    return (f(x+d) - f(x)) / d

x=1
d= 0.01

print(der_func(f,x, d))     

print(derivative(f,x, d))   


der_func_list=[]
derivative_list=[]

for i in range(-4, -16, -2):
    
    d= 10**i
    
    der_func_list.append(der_func(f,x,d))      
    
    derivative_list.append(derivative(f,x,d))
    
    
print(der_func_list)

print(derivative_list)

print("The more the delta get smaller the more accuracy we achieve")

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
def f(x): 
    
    return math.sqrt(1-x**2)

N=100
Integral=0
x=-1

for i in range(N): 
    
    Integral= Integral + (2/N)*f(x)
    
    x = x + 2/N
    
print(Integral)

print(math.pi/2)

print("we obtained proper result which is close to the true value")